In [1]:
import numpy as np
import pandas as pd

prompt_df = pd.read_parquet("./data/metadata-large.parquet", columns=["prompt"])

In [2]:
import nltk
from nltk.corpus import words

english_dict = set(words.words())

In [3]:
filtered = prompt_df[prompt_df.prompt.apply(lambda x: all(word in english_dict for word in x.split()))]

In [10]:
# filter the prompt column for entries containing the word face, portrait, person, or human.
filtered_more = filtered.loc[filtered.prompt.str.contains("face|person|human", case=False, regex=True) & ~filtered.prompt.str.contains(
    "[\(\)\[\]\|]|fantasy|illustration|painting|art", case=False, regex=True), ]

# remove duplicate prompts
filtered_more = filtered_more.drop_duplicates()


In [22]:
print(filtered.shape)

filtered.prompt.head()

(428837, 1)


0     beautiful porcelain ivory fair face woman biom...
1     complex 3 d render hyper detailed ultra sharp ...
15    complex 3 d render hyper detailed ultra sharp ...
16    complex 3 d render hyper detailed ultra sharp ...
33    complex 3 d render hyper detailed ultra sharp ...
Name: prompt, dtype: object

In [12]:
cohere_prompt = "Using the comments and posts made by this specific user, analyze their characteristics, personality, and word use to write me a prompt to generate one image of the person. The prompt should describe the user's face, facial hair, body, emotions, what they are doing, and their surroundings.\n\nThe prompt:"


In [19]:
downsampled = filtered.sample(n=50000)


# open text file
with open("./data/cohere_prompts.txt", "w") as f:
    # for each prompt in filtered, write to a text file
    for index, row in downsampled.iterrows():
        f.write(cohere_prompt + " " + row["prompt"] + "\n--\n")


In [14]:
df = pd.read_csv("./data/DALL-E_prompt_training - Sheet1.csv")

with open("./data/cohere_prompts.txt", "w") as f:
    # for each prompt in filtered, write to a text file
    for index, row in df.iterrows():
        f.write(cohere_prompt + " " + row["Prompt"] + "\n--\n")